In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# The data 

In [2]:
DATA_DIR = '/Users/fletchercollis/Desktop/Machine learning for behavioral data/PROJECT_GoGYMi/GoGymi' ### put your path here

activity = pd.read_csv('{}/activity.csv'.format(DATA_DIR))


students = pd.read_csv('{}/students.csv'.format(DATA_DIR))
teachers = pd.read_csv('{}/teachers.csv'.format(DATA_DIR))


gymitrainer = pd.read_csv('{}/gymitrainer.csv'.format(DATA_DIR))


math_questions = pd.read_csv('{}/math_questions.csv'.format(DATA_DIR)) ### NOT USEFUL HERE YET SO NOT LOOKED AT OR CLEANED
text_questions = pd.read_csv('{}/text_questions.csv'.format(DATA_DIR)) ### NOT USEFUL HERE YET SO NOT LOOKED AT OR CLEANED
#texts = pd.read_csv('{}/texts.csv'.format(DATA_DIR)) ### NOT USEFUL HERE YET SO NOT LOOKED AT OR CLEANED


math_results = pd.read_csv('{}/math_results.csv'.format(DATA_DIR))
essay_results = pd.read_csv('{}/essay_results.csv'.format(DATA_DIR))
text_results = pd.read_csv('{}/text_results.csv'.format(DATA_DIR))

all_scores = pd.read_csv('{}/all_scores.csv'.format(DATA_DIR))

In [3]:
activity['time_spent'] = (activity['activity_updated'] - activity['activity_started'])

activity[activity['time_spent'] <= 0].shape[0]

31816

# First, lets clean up the time in every dataframe to make it easier to manipulate!

In [ ]:
activity['activity_started'] = pd.to_datetime(activity['activity_started'], unit='s')
activity['activity_completed'] = pd.to_datetime(activity['activity_completed'], unit='s')
activity['activity_updated'] = pd.to_datetime(activity['activity_updated'], unit='s')

students['creation_time'] = pd.to_datetime(students['creation_time'], unit='s')
teachers['creation_time'] = pd.to_datetime(teachers['creation_time'], unit='s')

gymitrainer['start_time'] = pd.to_datetime(gymitrainer['startTime'], unit='s')
gymitrainer['end_time'] = pd.to_datetime(gymitrainer['endTime'], unit='s')

math_results['time'] = pd.to_datetime(math_results['time'], unit='s')
essay_results['time'] = pd.to_datetime(essay_results['time'], unit='s')
text_results['time'] = pd.to_datetime(text_results['time'], unit='s')

all_scores['time'] = pd.to_datetime(all_scores['time'], unit='s')

# Cleaning the user data

## activity dataframe 

In [ ]:
activity_og = activity.copy()

### Lets look at missing or problematic data 

- First lets see if there are any missing or problematic values and also drop "Unnamed: 0" column since it is not usefeul

In [ ]:
activity = activity.drop('Unnamed: 0', axis=1)  

activity.isnull().sum().reset_index(name='Nb of NAN')

- Only column where we have NAN and so we replace them with the last updated time and add a new column where 0 if uncompleted and 1 if completed

In the activity dataset in the activity_completed column, 0 or missing value both mean that the activity was never completed which either occurs if it was abandoned, or if the activity does not record when it is completed which is the case for access and exams from course ids 3301 and 5447 (these are related to the essays).

---> look into this 

In [ ]:
activity['Completed'] = activity['activity_completed'].notna().astype(int)

## Quick check column is correct :
print(activity_og.shape[0] - activity['Completed'].sum())

In [ ]:
rows_where_updated_diff_from_completed_without_nan = activity[activity['activity_completed'] != activity['activity_updated']].shape[0] - (activity_og.shape[0] - activity['Completed'].sum())

print(rows_where_updated_diff_from_completed_without_nan)

### Updated time is the same as completed time except in 33933 cases when we take out the NAN this leaves : 

In [ ]:
activity.loc[:, 'activity_completed'] = activity['activity_completed'].fillna(activity['activity_updated'])  

In [ ]:
activity.head()

- Also we notice that some of the activity_completed column are from 1970-01-01 and so we replace them with the last updated time 

----> do we consider these completed or not? I consider them completed just that there was a bug

In [ ]:
nb_of_unfeasible_dates = activity[activity['activity_completed'] < '2000-01-01'].shape[0]

print(rows_where_updated_diff_from_completed_without_nan - nb_of_unfeasible_dates)

### These also account for the majority of times the updated date is different from the completed one meaning we can assume updated = completed 

### (155 rows where this is not the case is negligeable for a dataset of 80000 rows)

In [ ]:
activity.loc[activity['activity_completed'] < '2000-01-01', 'activity_completed'] = activity['activity_updated']

### Lets check the time spent on various activities

- Now we look at the time spent on activities

In [ ]:
activity['time_spent'] = (activity['activity_updated'] - activity['activity_started']).dt.total_seconds()

activity.shape[0]

In [ ]:
activity['time_spent'].describe()

In [ ]:
data = activity['time_spent'].copy()

fig, axes = plt.subplots(1, 2, figsize=(9, 4)) 


axes[0].hist(data, bins=100, color='teal', edgecolor='black', alpha=0.7)
axes[0].set_title('Histogram of Time Spent')
axes[0].set_xlabel('Time Spent')
axes[0].set_ylabel('Frequency')

sns.boxplot(x=data, color='blue', ax=axes[1])
axes[1].set_title('Box Plot of Time Spent')
axes[1].set_xlabel('Time Spent (seconds)')

plt.tight_layout()
plt.show()

In [ ]:
print('Nb of rows where the length of the activity is negative or zero:', activity[activity['time_spent'] < 0].shape[0])

- Since there are few we decided to remove the problematic rows 

In [ ]:
activity = activity[activity['time_spent'] >= 0]

- How much data did we lose?

In [ ]:
activity_data_loss = activity.shape[0]/activity_og.shape[0]
print('Percent of original data lost', 100 - activity_data_loss * 100)

#### THIS IS ALOT SO MAYBE THINK ABOUT ALTERNATIVES?

- Now lets look at the distribution of time spent on activities

In [ ]:
data = activity['time_spent']

plt.hist(data, bins=100, color='teal', edgecolor='black', alpha=0.7)
plt.title('Histogram Example')
plt.xlabel('time_spent Values')
plt.ylabel('Frequency')
plt.show()

- There seems to be some outliers, so lets remove some activities that seem to be taking too long -----> NEED TO DECIDE WHAT TOO LONG IS 

In [ ]:
activity.groupby(by='activity_type')['time_spent'].describe()

In [ ]:
quantiles = activity.groupby(by='activity_type')['time_spent'].quantile(0.95)

activity = activity[activity.apply(lambda row: row['time_spent'] <= quantiles[row['activity_type']], axis=1)]

- Now lets's see how much data deleted overall :

In [ ]:
activity_data_loss = activity.shape[0]/activity_og.shape[0]
print('Percent of data lost in total', 100 - activity_data_loss * 100)

### Lets seperate the data out into Math, Text and Essay

Difference between "topic", "lesson" and "course" in activity.csv: a course is the highest level on the site. There are 8 courses of which a user will normally do 3-4. Inside of those courses are lessons which then have topics under them.

course ids 3301 and 5447 (these are related to the essays)

In [ ]:
activity['course_id'].unique()

In [ ]:
activity['course_id'].value_counts()

In [ ]:
math_activity = ...
essay_activity = ...
text_activity = ...

## Cleaning the test data

### all_scores datframe

In [ ]:
all_scores_og = all_scores.copy()

- First lets see if there are any missing or problematic values

In [ ]:
all_scores.isnull().sum().reset_index(name='Nb of NAN')

In [ ]:
all_scores[all_scores['time'] < '2023-01-01']

- No problematic values so now let's see if some users redid the same exam more than once

In [ ]:
attempt_count = all_scores.groupby(['user_id', 'test_id','course']).size().reset_index(name='attempt_count')

attempt_count[attempt_count['attempt_count'] > 1].shape[0]

- Let's get rid of everything past the first attempt ----> HERE DISCUSS

In [ ]:
all_scores = all_scores.sort_values(by='time')

all_scores = all_scores.groupby(['user_id', 'test_id', 'course']).first().reset_index()

- Now lets's see how much data deleted overall :

In [ ]:
all_scores_data_loss = all_scores.shape[0]/all_scores_og.shape[0]
print('Percent of data lost', 100 - all_scores_data_loss * 100)

- We also decided to round the "percentage" column to two decimal places to make life easier 

In [ ]:
all_scores['percentage'] = all_scores['percentage'].round(2)

### Lets seperate the data out into Math, Text and Essay

course ids 3301 and 5447 (these are related to the essays)

In [ ]:
all_scores

In [ ]:
all_scores['course'].value_counts()

In [ ]:
math_scores = ...
essay_scores = ...
text_scores = ...

# Now let's export our cleaned data !

Here will have 6 dataframes so far once seperated into the various subjects

In [ ]:
# Uncomment this to save the cleaned dataframes :)
   
# math_scores.to_csv('math_scores.csv', index=False)
# essay_scores.to_csv('essay_scores.csv', index=False)
# text_scores.to_csv('text_scores.csv', index=False)    

# math_activity.to_csv('math_activity.csv', index=False)
# essay_activity.to_csv('essay_activity.csv', index=False)
# text_activity.to_csv('text_activity.csv', index=False)    